In [1]:
import numpy as np
import pandas as pd
import random
import string
import tensorflow as tf
from tensorflow import keras
import tensorboard
import os
from pathlib import Path
import datetime

from sklearn.model_selection import ParameterSampler
from sklearn.model_selection import train_test_split

print('TensorFlow version: ', tf.__version__)
print('Keras version: ', keras.__version__)
print('Tensorboard version:', tensorboard.__version__)



TensorFlow version:  2.0.0
Keras version:  2.2.4-tf
Tensorboard version: 2.0.0


In [25]:
df = pd.read_csv(data_path / file_csv[0],)
df = df.drop(['model'],axis=1,inplace=False)
df['age'] = df['age']/100.0
df['dist'] = df['dist']/25000.0
df['year'] = df['year']/2020.0
print(df['dist'].max())
df.head()
df

0.99988


,age,city,dist,gender,make,year,new_used,quote
0,0.76,Norfolk County,0.66988,f,toyota,0.990594,used,410.040033
1,0.62,Kelowna,0.75080,f,acura,0.992574,used,431.022626
2,0.41,Quebec City,0.91720,f,honda,0.995050,used,410.040033
3,0.80,Gatineau,0.77396,f,acura,0.997030,used,421.946006
4,0.80,Blainville,0.77136,m,chevy,0.993069,used,720.832348
...,...,...,...,...,...,...,...,...
19995,0.83,Delta,0.34192,m,chevy,0.999010,used,720.832348
19996,0.79,Red Deer,0.59248,f,nissan,0.999505,new,410.040033
19997,0.27,Longueuil,0.52980,f,ford,0.990099,used,1744.513412
19998,0.71,Calgary,0.90784,m,mercedes,0.992574,used,773.543820


In [5]:
data_path = Path('/home/tim/Documents/flask_experiment/data')

# file_csv = []
# index_count = 0
# for file in os.listdir(data_path):
#     # open up the .mat file
#     if file.endswith(".csv"):
#         file_csv.append(file)
#         print(file)
#         df = pd.read_csv(data_path / file,)
#         # scale some of the large values
#         df['age'] = df['age']/100.0
#         df['dist'] = df['dist']/25000.0
#         df['year'] = df['year']/2020.0
#         df = df.drop(['model'],axis=1,inplace=False)
#         if index_count == 0:
#             y = df['quote'].to_numpy()
#             df = df.drop(['quote'],axis=1,inplace=False)
#             df = pd.get_dummies(df,drop_first=True)
#             X = df.to_numpy()
#             index_count += 1
#         else:
#             a = df['quote'].to_numpy()
#             y = np.vstack((a, y))
#             df = df.drop(['quote'],axis=1,inplace=False)
#             df = pd.get_dummies(df,drop_first=True)
#             X = np.dstack((df.to_numpy(), X))
#             index_count += 1

# y = np.reshape(y,(-1))
# X = np.reshape(X,(X.shape[0],X.shape[1],-1))
# print(X.shape)
# print(y.shape)
# print(file_csv)


file_csv = []
index_count = 0
for file in os.listdir(data_path):
    # open up the .mat file
    if file.endswith(".csv"):
        file_csv.append(file)
        print(file)
        df = pd.read_csv(data_path / file,)
        # scale some of the large values
        df['age'] = df['age']/100.0
        df['dist'] = df['dist']/25000.0
        df['year'] = df['year']/2020.0
        df = df.drop(['model'],axis=1,inplace=False)
        if index_count == 0:
            y = df['quote'].to_numpy()
            print(y.shape)
            df = df.drop(['quote'],axis=1,inplace=False)
            df = pd.get_dummies(df,drop_first=True)
            X = df.to_numpy()
            index_count += 1
        else:
            a = df['quote'].to_numpy()
            y = np.vstack((a, y))
            df = df.drop(['quote'],axis=1,inplace=False)
            df = pd.get_dummies(df,drop_first=True)
            X = np.dstack((df.to_numpy(), X))
            index_count += 1

X = np.reshape(X,(X.shape[0],-1,X.shape[1]))
print(X.shape)
print(y.shape)
print(file_csv)



company_1.csv
(20000,)
(20000, 1, 114)
(20000,)
['company_1.csv']


In [13]:
# split up the X and y for both the confidence interval and 
X_quote = X[0:int(X.shape[0]/2)]
y_quote = y[0:int(y.shape[0]/2)]
print(X_quote.shape)
print(y_quote.shape)
X_conf = X[int(X.shape[0]/2):]
y_conf = y[int(y.shape[0]/2):]
print(X_conf.shape)
print(y_conf.shape)

X_train_q, X_test_q, y_train_q, y_test_q = train_test_split(X_quote, y_quote, test_size=0.33, random_state=42)
X_train_q, X_val_q, y_train_q, y_val_q = train_test_split(X_train_q, y_train_q, test_size=0.33, random_state=42)

print("X_train_q shape:", X_train_q.shape, "\ty_train_q shape:", y_train_q.shape)
print("X_val_q:", X_val_q.shape, "\t\t\ty_val_q shape:", y_val_q.shape)
print("X_test_q:", X_test_q.shape, "\t\ty_test_q shape:", y_test_q.shape)



(10000, 1, 114)
(10000,)
(10000, 1, 114)
(10000,)
X_train_q shape: (4489, 1, 114) 	y_train_q shape: (4489,)
X_val_q: (2211, 1, 114) 			y_val_q shape: (2211,)
X_test_q: (3300, 1, 114) 		y_test_q shape: (3300,)


In [14]:
model = keras.models.Sequential([
    keras.layers.Dense(200, activation="relu", input_shape=[1, 114]),
    keras.layers.Dense(200, activation="relu"),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(1, activation="linear")
])

In [15]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 1, 200)            23000     
_________________________________________________________________
dense_9 (Dense)              (None, 1, 200)            40200     
_________________________________________________________________
dense_10 (Dense)             (None, 1, 50)             10050     
_________________________________________________________________
dense_11 (Dense)             (None, 1, 1)              51        
Total params: 73,301
Trainable params: 73,301
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(loss="mse",
              optimizer="adam",
              metrics=["mse"])

# create a name for the model so that we can track it in tensorboard
log_dir="logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "_quote"

# create tensorboard callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0,
                                                      update_freq='epoch',profile_batch=0)

earlystop_callback = tf.keras.callbacks.EarlyStopping(
        monitor="mse",
        patience=1000,
        restore_best_weights=True,
        verbose=1,)

In [ ]:
history = model.fit(X_train_q, y_train_q, 
                    epochs=10000,
                    verbose=1,
                    validation_data=(X_val_q,y_val_q),
                    callbacks=[tensorboard_callback, earlystop_callback])

Train on 4489 samples, validate on 2211 samples
Epoch 1/10000
4489/4489 [==============================] - 1s 248us/sample - loss: 13987.2987 - mse: 13987.2910 - val_loss: 2436.4410 - val_mse: 2436.4414
Epoch 2/10000
4489/4489 [==============================] - 1s 112us/sample - loss: 1225.4847 - mse: 1225.4843 - val_loss: 799.0053 - val_mse: 799.0054
Epoch 3/10000
4489/4489 [==============================] - 0s 104us/sample - loss: 757.2209 - mse: 757.2207 - val_loss: 689.8092 - val_mse: 689.8092
Epoch 4/10000
4489/4489 [==============================] - 1s 119us/sample - loss: 667.6405 - mse: 667.6404 - val_loss: 659.7022 - val_mse: 659.7022
Epoch 5/10000
4489/4489 [==============================] - 0s 111us/sample - loss: 617.5228 - mse: 617.5228 - val_loss: 610.8512 - val_mse: 610.8513
Epoch 6/10000
4489/4489 [==============================] - 0s 104us/sample - loss: 542.5860 - mse: 542.5861 - val_loss: 568.6434 - val_mse: 568.6434
Epoch 7/10000
4489/4489 [=========================

Epoch 56/10000
4489/4489 [==============================] - 0s 98us/sample - loss: 33.8660 - mse: 33.8660 - val_loss: 197.9223 - val_mse: 197.9223
Epoch 57/10000
4489/4489 [==============================] - 0s 101us/sample - loss: 33.1663 - mse: 33.1663 - val_loss: 200.4644 - val_mse: 200.4644
Epoch 58/10000
4489/4489 [==============================] - 0s 100us/sample - loss: 28.1669 - mse: 28.1669 - val_loss: 184.9489 - val_mse: 184.9489
Epoch 59/10000
4489/4489 [==============================] - 0s 106us/sample - loss: 26.9050 - mse: 26.9050 - val_loss: 175.5799 - val_mse: 175.5800
Epoch 60/10000
4489/4489 [==============================] - 0s 100us/sample - loss: 25.3985 - mse: 25.3985 - val_loss: 180.4669 - val_mse: 180.4669
Epoch 61/10000
4489/4489 [==============================] - 1s 125us/sample - loss: 25.8575 - mse: 25.8575 - val_loss: 195.8269 - val_mse: 195.8269
Epoch 62/10000
4489/4489 [==============================] - 1s 112us/sample - loss: 24.0205 - mse: 24.0205 - val_

4489/4489 [==============================] - 0s 97us/sample - loss: 27.2183 - mse: 27.2183 - val_loss: 147.0402 - val_mse: 147.0402
Epoch 112/10000
4489/4489 [==============================] - 0s 98us/sample - loss: 17.9944 - mse: 17.9944 - val_loss: 158.0296 - val_mse: 158.0296
Epoch 113/10000
4489/4489 [==============================] - 0s 98us/sample - loss: 13.2623 - mse: 13.2623 - val_loss: 141.8335 - val_mse: 141.8336
Epoch 114/10000
4489/4489 [==============================] - 0s 96us/sample - loss: 10.0325 - mse: 10.0325 - val_loss: 157.9731 - val_mse: 157.9730
Epoch 115/10000
4489/4489 [==============================] - 0s 97us/sample - loss: 13.6800 - mse: 13.6800 - val_loss: 138.8596 - val_mse: 138.8596
Epoch 116/10000
4489/4489 [==============================] - 0s 101us/sample - loss: 8.5500 - mse: 8.5500 - val_loss: 146.5641 - val_mse: 146.5640
Epoch 117/10000
4489/4489 [==============================] - 0s 97us/sample - loss: 8.9903 - mse: 8.9903 - val_loss: 153.6999 - v

Epoch 167/10000
4489/4489 [==============================] - 1s 223us/sample - loss: 8.8891 - mse: 8.8891 - val_loss: 146.5553 - val_mse: 146.5553
Epoch 168/10000
4489/4489 [==============================] - 1s 154us/sample - loss: 6.9858 - mse: 6.9858 - val_loss: 155.0044 - val_mse: 155.0044
Epoch 169/10000
4489/4489 [==============================] - 0s 98us/sample - loss: 7.1805 - mse: 7.1805 - val_loss: 166.7110 - val_mse: 166.7110
Epoch 170/10000
4489/4489 [==============================] - 0s 100us/sample - loss: 8.6452 - mse: 8.6452 - val_loss: 153.7237 - val_mse: 153.7236
Epoch 171/10000
4489/4489 [==============================] - 0s 100us/sample - loss: 11.2187 - mse: 11.2187 - val_loss: 155.2209 - val_mse: 155.2209
Epoch 172/10000
4489/4489 [==============================] - 0s 99us/sample - loss: 8.5174 - mse: 8.5174 - val_loss: 170.6007 - val_mse: 170.6007
Epoch 173/10000
4489/4489 [==============================] - 0s 98us/sample - loss: 13.7795 - mse: 13.7795 - val_loss:

Epoch 223/10000
4489/4489 [==============================] - 0s 103us/sample - loss: 5.9819 - mse: 5.9819 - val_loss: 168.6190 - val_mse: 168.6189
Epoch 224/10000
4489/4489 [==============================] - 0s 100us/sample - loss: 6.6716 - mse: 6.6716 - val_loss: 169.0451 - val_mse: 169.0451
Epoch 225/10000
4489/4489 [==============================] - 0s 100us/sample - loss: 11.7495 - mse: 11.7495 - val_loss: 179.4730 - val_mse: 179.4730
Epoch 226/10000
4489/4489 [==============================] - 0s 94us/sample - loss: 11.8449 - mse: 11.8449 - val_loss: 181.7744 - val_mse: 181.7744
Epoch 227/10000
4489/4489 [==============================] - 0s 96us/sample - loss: 15.5015 - mse: 15.5015 - val_loss: 177.7090 - val_mse: 177.7090
Epoch 228/10000
4489/4489 [==============================] - 0s 94us/sample - loss: 7.9850 - mse: 7.9850 - val_loss: 177.9735 - val_mse: 177.9734
Epoch 229/10000
3872/4489 [========================>.....] - ETA: 0s - loss: 6.1006 - mse: 6.1006

In [188]:
print(y.shape)
print(X.shape)

(5000, 5)
(5000, 114, 5)


In [76]:
y = np.reshape(y,(-1,5))

In [77]:
y.shape

(5000, 5)

In [32]:
df = df.drop(['quote'],axis=1,inplace=True)

In [33]:
df.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [44]:
df1 = pd.read_csv(data_path / file_csv[0],)
df1.head()

,age,city,dist,gender,make,year,model,new_used,quote
0,61,Belleville,10726,m,audi,2017,e-tron,used,352.599790
1,21,Wood Buffalo,9401,m,toyota,2001,camry,used,1723.855245
2,83,Kelowna,9427,m,acura,2016,mdx,used,221.792349
3,28,St. John,21090,m,ford,2003,explorer,used,1723.855245
4,56,Sarnia,7099,m,audi,2018,e-tron,used,354.748787


In [45]:
df1 = df1.drop(['quote'], axis=1, inplace=False)

In [46]:
X = df1.to_numpy()

In [47]:
X

array([[61, 'Belleville', 10726, ..., 2017, 'e-tron', 'used'],
       [21, 'Wood Buffalo', 9401, ..., 2001, 'camry', 'used'],
       [83, 'Kelowna', 9427, ..., 2016, 'mdx', 'used'],
       ...,
       [43, 'Montreal', 2619, ..., 2003, 'corrola', 'used'],
       [37, 'Laval', 16457, ..., 2017, 'c', 'used'],
       [20, 'Fredericton', 14860, ..., 2015, 's', 'used']], dtype=object)

In [15]:
df1 = pd.get_dummies(df1,drop_first=True)

In [16]:
df1.head()

,age,dist,year,quote,city_Airdrie,city_Ajax,city_Aurora,city_Barrie,city_Belleville,city_Blainville,...,model_mirano,model_r8,model_rsx,model_s,model_sierra,model_terrain,model_x,model_x5,model_yukon,new_used_used
0,61,10726,2017,352.599790,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,21,9401,2001,1723.855245,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,83,9427,2016,221.792349,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,28,21090,2003,1723.855245,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,56,7099,2018,354.748787,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [17]:
X1 = df1.to_numpy()
X1.shape

(5000, 140)